In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import random
import seaborn as sns
import math
import re
%matplotlib inline
pd.set_option('display.max_columns', 100)

In [2]:
df = pd.read_csv('spreadspoke_scores.csv')

In [3]:
# Drop all rows without Over/Under value 

df = df.dropna(subset=['over_under_line'])
df["spread_favorite"] = df['spread_favorite'] * -1
df["spread_favorite"] = np.where(df['spread_favorite'] == 0, 0, df['spread_favorite'])

In [4]:
# Add "Total" column with combined points of Home and Away team

df['total'] = df['score_home'] + df['score_away'] 
df = df.astype({'total': 'int'})

In [5]:
# Transform "Over/Under" Line to float

df = df[df.over_under_line != ' ']
df = df.astype({'over_under_line': 'float'})

In [6]:
# Add Target Variable 'Result' columns. Based on if Total Points when Over or Under the Over/Under Value

df['over_under_result'] = np.where(df['total'] > df['over_under_line'], 'Over', 'Under')

In [7]:
df

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,total,over_under_result
350,1/14/1968,1967,Superbowl,True,Green Bay Packers,33.0,14.0,Oakland Raiders,GB,13.5,43.0,Orange Bowl,True,60.0,12.0,74,NaN,47,Over
538,1/12/1969,1968,Superbowl,True,Baltimore Colts,7.0,16.0,New York Jets,IND,18.0,40.0,Orange Bowl,True,66.0,12.0,80,NaN,23,Under
727,1/11/1970,1969,Superbowl,True,Kansas City Chiefs,23.0,7.0,Minnesota Vikings,MIN,12.0,39.0,Tulane Stadium,True,55.0,14.0,84,NaN,30,Under
916,1/17/1971,1970,Superbowl,True,Baltimore Colts,16.0,13.0,Dallas Cowboys,IND,2.5,36.0,Orange Bowl,True,59.0,11.0,60,NaN,29,Under
1105,1/16/1972,1971,Superbowl,True,Dallas Cowboys,24.0,3.0,Miami Dolphins,DAL,6.0,34.0,Tulane Stadium,True,34.0,18.0,40,NaN,27,Under
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12792,11/1/2020,2020,8,False,Kansas City Chiefs,35.0,9.0,New York Jets,KC,19.5,49.0,Arrowhead Stadium,False,NaN,NaN,NaN,NaN,44,Under
12793,11/1/2020,2020,8,False,Miami Dolphins,28.0,17.0,Los Angeles Rams,LAR,3.5,45.5,Hard Rock Stadium,False,NaN,NaN,NaN,NaN,45,Under
12794,11/1/2020,2020,8,False,Philadelphia Eagles,23.0,9.0,Dallas Cowboys,PHI,11.5,42.5,Lincoln Financial Field,False,NaN,NaN,NaN,NaN,32,Under
12795,11/1/2020,2020,8,False,Seattle Seahawks,37.0,27.0,San Francisco 49ers,SEA,3.0,53.5,CenturyLink Field,False,NaN,NaN,NaN,NaN,64,Over


In [8]:
# Team Name Change without Location Change

df['team_home'] = np.where(df['team_home'] == 'Washington Redskins', 'Washington Football Team', df['team_home'])
df['team_away'] = np.where(df['team_away'] == 'Washington Redskins', 'Washington Football Team', df['team_away'])

In [9]:
# Change 'stadium' column name to 'stadium_name' to match Stadium Dataset

df['stadium_name'] = df['stadium']

# Stadium Name Changes

df['stadium_name'] = np.where(df['stadium_name'] == 'Joe Robbie Stadium', 'Hard Rock Stadium',df['stadium_name'])
df['stadium_name'] = np.where(df['stadium_name'] == 'Pro Player Stadium', 'Hard Rock Stadium',df['stadium_name'])
df['stadium_name'] = np.where(df['stadium_name'] == 'Dolphin Stadium', 'Hard Rock Stadium',df['stadium_name'])
df['stadium_name'] = np.where(df['stadium_name'] == 'Tampa Stadium', 'Raymond James Stadium',df['stadium_name'])
df['stadium_name'] = np.where(df['stadium_name'] == 'Alltel Stadium', 'TIAA Bank Field',df['stadium_name'])
df['stadium_name'] = np.where(df['stadium_name'] == 'Jack Murphy Stadium', 'Qualcomm Stadium',df['stadium_name'])

In [10]:
#Import Stadium Data

stadium_df = pd.read_csv('nfl_stadiums.csv')
print(stadium_df.shape)

(106, 15)


In [11]:
# Merge Stadium Data to Betting DataFrame

df = df.merge(stadium_df,on='stadium_name',how='left')

In [12]:
#Drop Uneeded Columns

df = df.drop(columns=['stadium','stadium_close','LATITUDE','LONGITUDE','NAME'])

In [13]:
# Manually Add Stadium Locations
# Drop Most Stadiums outside U.S.A.

df['stadium_location'] = np.where(df['stadium_name'] == 'FedEx Field', 'Landover, MD',df['stadium_location'])
df['stadium_location'] = np.where(df['stadium_name'] == 'TIAA Bank Field', 'Jacksonville, FL',df['stadium_location'])
df = df.dropna(subset=['stadium_location'])
df = df.drop([6168,9219,9484,10021,7118,7366,7576,7828,8203,8437,9158,9424,9435])

In [14]:
# Only Scores after 1978

df = df[df.schedule_season > 1978]

In [15]:
# Stadium Open Dates and Change to Int

df['stadium_open'] = np.where(df['stadium_name'] == 'FedEx Field', 1997,df['stadium_open'])
df['stadium_open'] = np.where(df['stadium_name'] == 'TIAA Bank Field', 1994,df['stadium_open'])
df['stadium_open'] = np.where(df['stadium_name'] == 'Rogers Centre', 1989,df['stadium_open'])
df['stadium_open'] = np.where(df['stadium_name'] == 'Rose Bowl', 1921,df['stadium_open'])
df['stadium_open'] = np.where(df['stadium_name'] == 'Tulane Stadium', 1926,df['stadium_open'])
df['stadium_open'] = np.where(df['stadium_name'] == 'Alamo Dome', 1993,df['stadium_open'])
df['stadium_open'] = np.where(df['stadium_name'] == 'Rice Stadium', 1950,df['stadium_open'])
df['stadium_open'] = np.where(df['stadium_name'] == 'Stanford Stadium', 1921,df['stadium_open'])

df = df.astype({'stadium_open': 'int'})

In [16]:
# Fix Stadium Type

df['stadium_type'] = np.where(df['stadium_name'] == 'FedEx Field', 'outdoor',df['stadium_type'])
df['stadium_type'] = np.where(df['stadium_name'] == 'TIAA Bank Field', 'outdoor',df['stadium_type'])
df['stadium_type'] = np.where(df['stadium_name'] == 'Stanford Stadium', 'outdoor',df['stadium_type'])

In [17]:
# Get Zipcodes

df['zipcode'] = df['stadium_address'].str[-5:]

# Adding Zip Codes to Stadiums Without Address

df['zipcode'] = np.where(df['stadium_name'] == 'Rose Bowl', 91103,df['zipcode'])
df['zipcode'] = np.where(df['stadium_name'] == 'Stanford Stadium', 94305,df['zipcode'])
df['zipcode'] = np.where(df['stadium_name'] == 'FedEx Field', 20785,df['zipcode'])
df['zipcode'] = np.where(df['stadium_name'] == 'TIAA Bank Field', 32202,df['zipcode'])
df['zipcode'] = np.where(df['stadium_name'] == 'Mercedes-Benz Stadium', 30313,df['zipcode'])
df['zipcode'] = np.where(df['stadium_name'] == 'SoFi Stadium', 90301,df['zipcode'])
df['zipcode'] = np.where(df['stadium_name'] == 'Allegiant Stadium', 89118,df['zipcode'])

#Outside U.S.

df['zipcode'] = np.where(df['stadium_name'] == 'Wembley Stadium', 11111,df['zipcode'])

# Zipcodes to Integers

df['zipcode'] = df['zipcode'].astype(int)

In [18]:
# Fix Stadium Surface

df['stadium_surface'] = np.where(df['stadium_surface'] == 'Hellas Matrix Turf', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_surface'] == 'Grass, Turf (1971-1974)', 'Grass',df['stadium_surface'])

df['stadium_surface'] = np.where(df['stadium_name'] == 'Giants Stadium', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Candlestick Park', 'Grass',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Sun Life Stadium', 'Grass',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Texas Stadium', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Hubert H. Humphrey Metrodome', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'RCA Dome', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Veterans Stadium', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'FedEx Field', 'Grass',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Foxboro Stadium', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Pontiac Silverdome', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Mile High Stadium', 'Grass',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Three Rivers Stadium', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Edward Jones Dome', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Cinergy Field', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Seattle Kingdome', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == "Houlihan's Stadium", 'Grass',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Houston Astrodome', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'RFK Memorial Stadium', 'Grass',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Cleveland Municipal Stadium', 'Grass',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Anaheim Stadium', 'Grass',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Atlanta-Fulton County Stadium', 'Grass',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Busch Memorial Stadium', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Orange Bowl', 'Grass',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Memorial Stadium (Baltimore)', 'Grass',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Sun Devil Stadium', 'Grass',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Mall of America Field', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Metropolitan Stadium', 'Grass',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Wembley Stadium', 'Grass',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_name'] == 'Husky Stadium', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = df[['stadium_surface']].fillna(value='Grass')




In [19]:
df.zipcode.value_counts()

7073     669
2035     353
15212    347
19148    346
80204    346
54304    342
64129    337
70112    336
45202    332
33607    331
14127    330
60605    329
92108    309
44114    307
46225    303
77054    299
55415    296
94124    294
33056    277
75062    243
30313    237
94621    224
85305    216
28202    203
32202    201
20785    191
21230    187
48342    181
37213    176
63101    168
98104    161
98134    159
48226    150
20003    142
90037    131
92806    123
30312     98
76011     97
63102     70
33125     60
95054     55
21218     52
85287     50
55425     24
11111     22
90746     22
98195     19
55455     18
61820     10
38104      8
37203      8
29634      8
90301      6
70803      4
91103      4
89118      3
78203      3
94305      1
Name: zipcode, dtype: int64

In [20]:
# Fix Stadium Capacity

df['stadium_capacity'] = np.where(df['stadium_name'] == 'Giants Stadium',80242,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Candlestick Park',69732,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Sun Life Stadium',64767,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Texas Stadium',65675,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Hubert H. Humphrey Metrodome', 64121,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'RCA Dome',60567,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Veterans Stadium',65352,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'FedEx Field', 82000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Foxboro Stadium',60292,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Pontiac Silverdome',80311,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Mile High Stadium', 75000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Three Rivers Stadium',59000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Edward Jones Dome', 67277,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Cinergy Field', 59754,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Seattle Kingdome',66000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == "Houlihan's Stadium", 50000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Houston Astrodome', 65000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'RFK Memorial Stadium',45596,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Cleveland Municipal Stadium', 81000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Anaheim Stadium', 69008,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Atlanta-Fulton County Stadium', 60606,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Busch Memorial Stadium', 60000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Orange Bowl', 75000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Memorial Stadium (Baltimore)', 50000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Sun Devil Stadium', 53599,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Mall of America Field', 64121,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Metropolitan Stadium', 41200,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Wembley Stadium', 86000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Husky Stadium', 70000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'TCF Bank Stadium', 50805,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'TIAA Bank Field', 67814,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == "Memorial Stadium (Champaign)", 60670,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == "Memorial Stadium (Clemson)", 74000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Liberty Bowl Memorial Stadium', 58325,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Vanderbilt Stadium', 40550,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Rose Bowl', 92542,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == "Tiger Stadium (LSU)", 100000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Tulane Stadium', 70000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Stanford Stadium', 50000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Rice Stadium', 47000,df['stadium_capacity'])


In [21]:
df[df['stadium_capacity'].isna()].stadium_name.value_counts()

Series([], Name: stadium_name, dtype: int64)

In [22]:
df.dtypes

schedule_date                    object
schedule_season                   int64
schedule_week                    object
schedule_playoff                   bool
team_home                        object
score_home                      float64
score_away                      float64
team_away                        object
team_favorite_id                 object
spread_favorite                 float64
over_under_line                 float64
stadium_neutral                    bool
weather_temperature             float64
weather_wind_mph                float64
weather_humidity                 object
weather_detail                   object
total                             int64
over_under_result                object
stadium_name                     object
stadium_location                 object
stadium_open                      int64
stadium_type                     object
stadium_address                  object
stadium_weather_station_code     object
stadium_weather_type             object


In [23]:
df

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,total,over_under_result,stadium_name,stadium_location,stadium_open,stadium_type,stadium_address,stadium_weather_station_code,stadium_weather_type,stadium_capacity,stadium_surface,STATION,ELEVATION,zipcode
12,9/1/1979,1979,1,False,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,TB,3.0,30.0,False,79.0,9.0,87,NaN,47,Over,Houlihan's Stadium,"Tampa, FL",1969,outdoor,"4201 North Dale Mabry Highway, Tampa, Florida ...",33607,warm,50000,Grass,USW00012842,5.8,33607
13,9/2/1979,1979,1,False,Buffalo Bills,7.0,9.0,Miami Dolphins,MIA,5.0,39.0,False,74.0,15.0,74,NaN,16,Under,Ralph Wilson Stadium,"Orchard Park, NY",1973,outdoor,"1 Bills Dr, Orchard Park, NY 14127",14127,cold,"73,967",FieldTurf,US1NYER0093,178.0,14127
14,9/2/1979,1979,1,False,Chicago Bears,6.0,3.0,Green Bay Packers,CHI,3.0,31.0,False,78.0,11.0,68,NaN,9,Under,Soldier Field,"Chicago, IL",1926,outdoor,"1410 Museum Campus Dr, Chicago, IL 60605",60605,cold,"61,500",Grass,USC00111550,177.7,60605
15,9/2/1979,1979,1,False,Denver Broncos,10.0,0.0,Cincinnati Bengals,DEN,3.0,31.5,False,69.0,6.0,38,NaN,10,Under,Mile High Stadium,"Denver, CO",1960,outdoor,"1701 Bryant St, Denver, CO 80204",80204,cold,75000,Grass,USW00023062,1611.2,80204
16,9/2/1979,1979,1,False,Kansas City Chiefs,14.0,0.0,Baltimore Colts,KC,1.0,37.0,False,76.0,8.0,71,NaN,14,Under,Arrowhead Stadium,"Kansas City, MO",1972,outdoor,"1 Arrowhead Dr, Kansas City, MO 64129",64129,cold,"76,416",Grass,US1MOJC0028,264.9,64129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,11/1/2020,2020,8,False,Kansas City Chiefs,35.0,9.0,New York Jets,KC,19.5,49.0,False,NaN,NaN,NaN,NaN,44,Under,Arrowhead Stadium,"Kansas City, MO",1972,outdoor,"1 Arrowhead Dr, Kansas City, MO 64129",64129,cold,"76,416",Grass,US1MOJC0028,264.9,64129
10242,11/1/2020,2020,8,False,Miami Dolphins,28.0,17.0,Los Angeles Rams,LAR,3.5,45.5,False,NaN,NaN,NaN,NaN,45,Under,Hard Rock Stadium,"Miami Gardens, FL",1987,outdoor,"347 Don Shula Dr, Miami Gardens, FL 33056",33056,warm,"65,326",Grass,USW00012839,8.8,33056
10243,11/1/2020,2020,8,False,Philadelphia Eagles,23.0,9.0,Dallas Cowboys,PHI,11.5,42.5,False,NaN,NaN,NaN,NaN,32,Under,Lincoln Financial Field,"Philadelphia, PA",2003,outdoor,"1020 Pattison Ave, Philadelphia, PA 19148",19148,cold,"69,176",Grass,USW00013739,3.0,19148
10244,11/1/2020,2020,8,False,Seattle Seahawks,37.0,27.0,San Francisco 49ers,SEA,3.0,53.5,False,NaN,NaN,NaN,NaN,64,Over,CenturyLink Field,"Seattle, WA",2002,outdoor,"800 Occidental Ave S, Seattle, WA 98134",98134,moderate,"67,000",FieldTurf,US1WAKG0038,93.0,98134


### Fixing Data Types

In [24]:
# converting the string to datetime format 
import datetime

df['schedule_date'] = pd.to_datetime(df['schedule_date'])

print(df.dtypes)

schedule_date                   datetime64[ns]
schedule_season                          int64
schedule_week                           object
schedule_playoff                          bool
team_home                               object
score_home                             float64
score_away                             float64
team_away                               object
team_favorite_id                        object
spread_favorite                        float64
over_under_line                        float64
stadium_neutral                           bool
weather_temperature                    float64
weather_wind_mph                       float64
weather_humidity                        object
weather_detail                          object
total                                    int64
over_under_result                       object
stadium_name                            object
stadium_location                        object
stadium_open                             int64
stadium_type 

In [25]:
# Capacity to Int

df['stadium_capacity'] = df.stadium_capacity.replace(',','', regex=True)
df = df.astype({'stadium_capacity': 'int'})

In [26]:
# Schedule Week 
# 1-17 Regular Season , > 17 Playoffs
# 18 = Wildcard
# 19 = Divisonal Round
# 20 = Conference Championship
# 21 = Superbowl

df['schedule_week'] = np.where(df['schedule_week'] == 'Division', 19,df['schedule_week'])
df['schedule_week'] = np.where(df['schedule_week'] == "Wildcard", 18,df['schedule_week'])
df['schedule_week'] = np.where(df['schedule_week'] == "WildCard", 18,df['schedule_week'])
df['schedule_week'] = np.where(df['schedule_week'] == 'Conference', 20,df['schedule_week'])
df['schedule_week'] = np.where(df['schedule_week'] == 'Superbowl', 21,df['schedule_week'])
df['schedule_week'] = np.where(df['schedule_week'] == 'SuperBowl', 21,df['schedule_week'])

df = df.astype({'schedule_week': 'int'})

In [27]:
df.dtypes

schedule_date                   datetime64[ns]
schedule_season                          int64
schedule_week                            int64
schedule_playoff                          bool
team_home                               object
score_home                             float64
score_away                             float64
team_away                               object
team_favorite_id                        object
spread_favorite                        float64
over_under_line                        float64
stadium_neutral                           bool
weather_temperature                    float64
weather_wind_mph                       float64
weather_humidity                        object
weather_detail                          object
total                                    int64
over_under_result                       object
stadium_name                            object
stadium_location                        object
stadium_open                             int64
stadium_type 

In [28]:
df.schedule_playoff.value_counts()

False    9845
True      373
Name: schedule_playoff, dtype: int64

## Add Team Data

In [89]:
team_df = pd.read_csv('teams_df.csv')

# Fix Outdated Team Names
team_df['Tm'] = np.where(team_df['Tm'] == 'Washington Redskins', 'Washington Football Team',team_df['Tm'])

In [90]:
home_team_df = team_df
away_team_df = team_df

In [91]:
# Get Home Team Season Data & Merge to Final Dataframe
home_team_df['schedule_season'] = home_team_df['Season_Year'] + 1 
home_team_df['T'] = home_team_df['T'].fillna(0)
home_team_df['PF_per_game'] = home_team_df['PF']/(home_team_df['W'] + home_team_df['L'] + home_team_df['T'])
home_team_df['PA_per_game'] = home_team_df['PA']/(home_team_df['W'] + home_team_df['L'] + home_team_df['T'])
home_team_df = home_team_df.drop(columns=['Season_Year', 'W', 'L', 'T','PF','PA'])

col_names = list(home_team_df.columns)
home_col_names = ["home_" + col for col in col_names]
home_team_df.columns = home_col_names
home_team_df['team_home'] = home_team_df['home_Tm']
home_team_df['schedule_season'] = home_team_df['home_schedule_season']
home_team_df = home_team_df.drop(columns=['home_Tm','home_schedule_season'])

final_df = df.merge(home_team_df,how='left')

In [92]:
# Get Away Team Season Data & Merge to Final Dataframe
away_team_df['schedule_season'] = away_team_df['Season_Year'] + 1
away_team_df['T'] = away_team_df['T'].fillna(0)
away_team_df['PF_per_game'] = away_team_df['PF']/(away_team_df['W'] + away_team_df['L'] + away_team_df['T'])
away_team_df['PA_per_game'] = away_team_df['PA']/(away_team_df['W'] + away_team_df['L'] + away_team_df['T'])
away_team_df = away_team_df.drop(columns=['Season_Year', 'W', 'L', 'T','PF','PA'])

col_names = list(away_team_df.columns)
away_col_names = ["away_" + col for col in col_names]
away_team_df.columns = away_col_names
away_team_df['team_home'] = away_team_df['away_Tm']
away_team_df['schedule_season'] = away_team_df['away_schedule_season']
away_team_df = away_team_df.drop(columns=['away_Tm','away_schedule_season'])

final_df = final_df.merge(away_team_df,how='left')

In [93]:
#### Will Amend if there is Time #### Fix Outdate Team Names

final_df[final_df['home_W-L%'].isna()].team_home.value_counts()

final_df = final_df.dropna(subset=['home_W-L%'])

In [95]:
final_df.to_csv('final_df.csv', index = False, header=True)

In [96]:
final_df

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,total,over_under_result,stadium_name,stadium_location,stadium_open,stadium_type,stadium_address,stadium_weather_station_code,stadium_weather_type,stadium_capacity,stadium_surface,STATION,ELEVATION,zipcode,home_W-L%,home_PD,home_MoV,home_SoS,home_SRS,home_OSRS,home_DSRS,home_PF_per_game,home_PA_per_game,away_W-L%,away_PD,away_MoV,away_SoS,away_SRS,away_OSRS,away_DSRS,away_PF_per_game,away_PA_per_game
0,1979-09-01,1979,1,False,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,TB,3.0,30.0,False,79.0,9.0,87,NaN,47,Over,Houlihan's Stadium,"Tampa, FL",1969,outdoor,"4201 North Dale Mabry Highway, Tampa, Florida ...",33607,warm,50000,Grass,USW00012842,5.8,33607,0.313,-18.0,-1.1,-2.2,-3.3,-4.1,0.8,15.0625,16.1875,0.313,-18.0,-1.1,-2.2,-3.3,-4.1,0.8,15.0625,16.1875
1,1979-09-02,1979,1,False,Buffalo Bills,7.0,9.0,Miami Dolphins,MIA,5.0,39.0,False,74.0,15.0,74,NaN,16,Under,Ralph Wilson Stadium,"Orchard Park, NY",1973,outdoor,"1 Bills Dr, Orchard Park, NY 14127",14127,cold,73967,FieldTurf,US1NYER0093,178.0,14127,0.313,-52.0,-3.3,0.0,-3.3,-0.1,-3.2,18.8750,22.1250,0.313,-52.0,-3.3,0.0,-3.3,-0.1,-3.2,18.8750,22.1250
2,1979-09-02,1979,1,False,Chicago Bears,6.0,3.0,Green Bay Packers,CHI,3.0,31.0,False,78.0,11.0,68,NaN,9,Under,Soldier Field,"Chicago, IL",1926,outdoor,"1410 Museum Campus Dr, Chicago, IL 60605",60605,cold,61500,Grass,USC00111550,177.7,60605,0.438,-21.0,-1.3,-1.1,-2.4,-2.7,0.3,15.8125,17.1250,0.438,-21.0,-1.3,-1.1,-2.4,-2.7,0.3,15.8125,17.1250
3,1979-09-02,1979,1,False,Denver Broncos,10.0,0.0,Cincinnati Bengals,DEN,3.0,31.5,False,69.0,6.0,38,NaN,10,Under,Mile High Stadium,"Denver, CO",1960,outdoor,"1701 Bryant St, Denver, CO 80204",80204,cold,75000,Grass,USW00023062,1611.2,80204,0.625,84.0,5.3,-0.2,5.0,-1.9,6.9,17.6250,12.3750,0.625,84.0,5.3,-0.2,5.0,-1.9,6.9,17.6250,12.3750
4,1979-09-02,1979,1,False,Kansas City Chiefs,14.0,0.0,Baltimore Colts,KC,1.0,37.0,False,76.0,8.0,71,NaN,14,Under,Arrowhead Stadium,"Kansas City, MO",1972,outdoor,"1 Arrowhead Dr, Kansas City, MO 64129",64129,cold,76416,Grass,US1MOJC0028,264.9,64129,0.250,-84.0,-5.3,0.9,-4.4,-3.0,-1.4,15.1875,20.4375,0.250,-84.0,-5.3,0.9,-4.4,-3.0,-1.4,15.1875,20.4375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10213,2020-11-01,2020,8,False,Kansas City Chiefs,35.0,9.0,New York Jets,KC,19.5,49.0,False,NaN,NaN,NaN,NaN,44,Under,Arrowhead Stadium,"Kansas City, MO",1972,outdoor,"1 Arrowhead Dr, Kansas City, MO 64129",64129,cold,76416,Grass,US1MOJC0028,264.9,64129,0.750,143.0,8.9,0.2,9.1,6.2,2.9,28.1875,19.2500,0.750,143.0,8.9,0.2,9.1,6.2,2.9,28.1875,19.2500
10214,2020-11-01,2020,8,False,Miami Dolphins,28.0,17.0,Los Angeles Rams,LAR,3.5,45.5,False,NaN,NaN,NaN,NaN,45,Under,Hard Rock Stadium,"Miami Gardens, FL",1987,outdoor,"347 Don Shula Dr, Miami Gardens, FL 33056",33056,warm,65326,Grass,USW00012839,8.8,33056,0.313,-188.0,-11.8,0.2,-11.6,-2.4,-9.1,19.1250,30.8750,0.313,-188.0,-11.8,0.2,-11.6,-2.4,-9.1,19.1250,30.8750
10215,2020-11-01,2020,8,False,Philadelphia Eagles,23.0,9.0,Dallas Cowboys,PHI,11.5,42.5,False,NaN,NaN,NaN,NaN,32,Under,Lincoln Financial Field,"Philadelphia, PA",2003,outdoor,"1020 Pattison Ave, Philadelphia, PA 19148",19148,cold,69176,Grass,USW00013739,3.0,19148,0.563,31.0,1.9,-1.7,0.3,0.7,-0.4,24.0625,22.1250,0.563,31.0,1.9,-1.7,0.3,0.7,-0.4,24.0625,22.1250
10216,2020-11-01,2020,8,False,Seattle Seahawks,37.0,27.0,San Francisco 49ers,SEA,3.0,53.5,False,NaN,NaN,NaN,NaN,64,Over,CenturyLink Field,"Seattle, WA",2002,outdoor,"800 Occidental Ave S, Seattle, WA 98134",98134,moderate,67000,FieldTurf,US1WAKG0038,93.0,98134,0.688,7.0,0.4,2.3,2.7,2.9,-0.2,25.3125,24.8750,0.688,7.0,0.4,2.3,2.7,2.9,-0.2,2